<a href="https://colab.research.google.com/github/tiagoandremv/MachineLearning/blob/main/FaceUNMASKER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de bibliotecas

In [1]:
# IMPORTAÇÃO DE BIBLIOTECAS NECESSÁRIAS
import os
import cv2
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, mean_squared_error, roc_auc_score, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers    import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models    import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses    import SparseCategoricalCrossentropy, CategoricalCrossentropy, binary_crossentropy

from random import shuffle
from tqdm   import tqdm # permite verificar o progresso do carregamento


class PrepareData:

  def __init__(self):
    path = '/content/drive/My Drive/Colab Notebooks/tdx_project/dataset-proj/'
    self.train_path = path + 'train/'  
    self.validation_path = path + 'validation/' 

  def label_img(self, _path):
    word_label = _path.split('/')[-2]
    return [1,0] if word_label == 'fake' else [0,1]

  def scale_image(self, img, scale_percent = 0.5):
    width = int(img.shape[1] * scale_percent)
    height = int(img.shape[0] * scale_percent)
    scale = (width, height)
    img = cv2.resize(img, scale)
    return img

  def load_prepare_data (self, _type):
    dataset = []
    _path = self.train_path if _type == 'train' else self.validation_path 
    
    for folder in os.listdir(_path): #['fake', 'real']
      path_folder = os.path.join(_path, folder)
      
      for filename in tqdm(os.listdir(path_folder)):
        path = os.path.join(path_folder, filename)
        label = self.label_img(path) # [0,1] para real , [1,0] para fake

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = self.scale_image(img)
        img = img.astype("float64") / 255.0
        #original_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
        
        dataset.append([np.array(img), np.array(label)])
    
    shuffle(dataset)
    return dataset    

def model_train_and_evaluate(model, trainX, trainY, validationX, validationY, key=""):
  model = model.fit(trainX, trainY)
  yproba = model.predict_proba(validationX)[::,1]
  ypredicted = model.predict(validationX)
  
  fpr, tpr, _ = roc_curve(validationY,  yproba)
  auc         = roc_auc_score(validationY, yproba)
  conf_m      = confusion_matrix(validationY, ypredicted)
  mcc         = matthews_corrcoef(validationY, ypredicted)
  f1          = f1_score(validationY, ypredicted)
  result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc','f1', 'mcc', 'confusion_matrix'])
  result_table = result_table.append({'classifiers':model.__class__.__name__+"-"+key,
                                      'fpr':fpr, 
                                      'tpr':tpr, 
                                      'auc':auc,
                                      'confusion_matrix': conf_m,
                                      'mcc':mcc,
                                      'f1': f1}, ignore_index=True)
  return result_table


def visualize_fit_summary(hist, n_epochs):
  acc = hist.hist['accuracy']
  val_acc = hist.hist['val_accuracy']

  loss = hist.hist['loss']
  val_loss = hist.hist['val_loss']

  epochs_range = range(n_epochs)

  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()

def plot_all_images(images):
  for i in range(len(images)):
    plt.subplot(1,len(images),i+1) 
    plt.imshow(images[i])
    plt.title(titles[i])
    plt.xticks([])
    plt.yticks([])
  plt.show()

#Carregamento de Imagens

In [2]:
pData = PrepareData()
train_data = pData.load_prepare_data(_type = 'train')
print("train_data Shape: ", np.asarray(train_data).shape)

100%|██████████| 700/700 [00:43<00:00, 16.14it/s]

train_data Shape:  (1400, 2)


In [3]:
validation_data = pData.load_prepare_data(_type = 'validation')
print("validation_data Shape: ", np.asarray(validation_data).shape)


100%|██████████| 300/300 [00:18<00:00, 16.24it/s]

validation_data Shape:  (600, 2)


#Definição de features

In [4]:
# X-Features & Y-Labels
train_images = np.array([i[0] for i in train_data]).reshape(-1, 512, 512, 1)
train_labels = np.array([i[1] for i in train_data]).astype('float64')

validation_images = np.array([i[0] for i in validation_data]).reshape(-1, 512, 512, 1)
validation_labels = np.array([i[1] for i in validation_data]).astype('float64')

print("Train Images, shape and type: ", train_images.shape, train_images.dtype)
print("Train Labels, shape and type: ", train_labels.shape, train_labels.dtype)

print("Validation Images, shape and type: ", validation_images.shape, validation_images.dtype)
print("Validation Labels, shape and type: ", validation_labels.shape, validation_labels.dtype)

Train Images, shape and type:  (1400, 512, 512, 1) float64
Train Labels, shape and type:  (1400, 2) float64
Validation Images, shape and type:  (600, 512, 512, 1) float64
Validation Labels, shape and type:  (600, 2) float64


In [5]:
train_images[1].max()

0.9137254901960784

# Definição do MODEL

In [6]:
early_callback = EarlyStopping(monitor = 'loss', patience = 2)
inputs_shape = validation_images[0].shape
print("Inputs Shape: ", inputs_shape)

model = Sequential()
model.add(Conv2D(16, 2, activation='relu', padding='same', input_shape=inputs_shape))
model.add(MaxPooling2D())
model.add(Conv2D(32, 2, activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 2, activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2))


model.compile(optimizer='adam', 
              loss=CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Inputs Shape:  (512, 512, 1)


In [ ]:
history = model.fit(train_images, 
                    train_labels, 
                    epochs = 30, 
                    validation_data = (validation_images, validation_labels), 
                    callbacks = [early_callback] 
                    )

Epoch 1/30


In [ ]:
model.summary()

In [ ]:
visualize_fit_summary(history, 30)

In [ ]:
# APRESENTAÇÃO DE RESULTADOS

#print("Confusion Matrix\n", confusion_matrix(y_train, train_predictions))
#print("Accuracy: %.2f %%" % (accuracy_score(y_train, train_predictions) * 100))